In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20, 4

In [2]:
from AJ_ML_Library import models_classification

In [3]:
from sklearn.preprocessing import LabelEncoder

In [4]:
train = pd.read_csv('data/train_modified_10.csv')
test = pd.read_csv('data/test_modified_10.csv')

In [5]:
train.shape, test.shape

((7381, 232), (11171, 231))

In [6]:
train.columns

Index([u'event_count', u'event_type 10', u'event_type 11', u'event_type 13',
       u'event_type 14', u'event_type 15', u'event_type 18', u'event_type 2',
       u'event_type 20', u'event_type 21',
       ...
       u'resource_type 6', u'resource_type 7', u'resource_type 8',
       u'resource_type 9', u'severity_type 1', u'severity_type 2',
       u'severity_type 3', u'severity_type 4', u'severity_type 5',
       u'location_occurence_count'],
      dtype='object', length=232)

In [7]:
train['fault_severity'].value_counts()

0    4784
1    1871
2     726
Name: fault_severity, dtype: int64

In [8]:
target = 'fault_severity'
IDcol = 'id'
output_colnames = ['predict_0','predict_1','predict_2']

In [9]:
#Combine the data first:
train['source'] = 'train'
test['source'] = 'test'
data = pd.concat([train,test], ignore_index=True)
data.shape

(18552, 233)

## Variable for Resource Type:

In [11]:
data['resource_type 8and2'] = data.apply(lambda x: 1 if (x['resource_type 2']+x['resource_type 8'])==2 else 0,axis=1)
data['resource_type 8and2'].value_counts()

0    17612
1      940
Name: resource_type 8and2, dtype: int64

In [12]:
def combine_8_2(x):
    s = x['resource_type 2']+x['resource_type 8']
    
    if s==2:
        return 'both'
    elif s==0:
        return 'none'
    else:
        if x['resource_type 2']==1:
            return 'only 2'
        else:
            return 'only 8'
        
data['resource_type 82combo'] = data.apply(combine_8_2, axis=1)
data['resource_type 82combo'].value_counts()

only 8    9328
only 2    7978
both       940
none       306
Name: resource_type 82combo, dtype: int64

In [13]:
data['resource_type 82combo_lag'] = data['resource_type 82combo'].shift().fillna(-1)
data['resource_type_change'] = data.apply(lambda x: 1 if x['resource_type 82combo']==x['resource_type 82combo_lag'] else 0,axis=1)
data.drop(['resource_type 82combo','resource_type 82combo_lag'],axis=1,inplace=True)

In [14]:
data['resource_type_change'].value_counts()

0    10286
1     8266
Name: resource_type_change, dtype: int64

## Check if previous location is same:

In [15]:
data['location_lag'] = data['location'].shift().fillna(-1)
data['location_previous_same'] = data.apply(lambda x: 1 if x['location']==x['location_lag'] else 0,axis=1)
data.drop(['location_lag'],axis=1,inplace=True)

## Divide back into test and train

In [48]:
train = data.loc[data['source']=='train']
test = data.loc[data['source']=='test']
train.drop('source',axis=1,inplace=True)
test.drop(['source','fault_severity'],axis=1,inplace=True)
train.to_csv('data/train_modified_11.csv',index=False)
test.to_csv('data/test_modified_11.csv',index=False)

/Users/aarshay/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/aarshay/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
